In [1]:
import os
import requests
import polars as pl
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
API_KEY = os.getenv("ALPHA_VANTAGE_API_KEY")
BASE_URL = "https://www.alphavantage.co/query"

symbol = "AAPL"  # Apple Inc. (Symbol)
function = "TIME_SERIES_DAILY"
params = {"function": function, "symbol": symbol, "apikey": API_KEY, "datatype": "json"}
response = requests.get(BASE_URL, params=params)

if response.status_code == 200:
    data = response.json()
else:
    print(f"Fehler beim Abrufen der Daten: {response.status_code}")
    exit()

In [3]:
try:
    time_series = data["Time Series (Daily)"]
except KeyError:
    print("Fehler: Die API-Antwort enthält keine Zeitseriendaten.")
    exit()

In [4]:
data_list = []
for date, values in time_series.items():
    data_list.append({
        "Date": date,
        "Open": float(values["1. open"]),
        "High": float(values["2. high"]),
        "Low": float(values["3. low"]),
        "Close": float(values["4. close"]),
        "Volume": int(values["5. volume"])
    })

In [5]:
raw_data = pl.DataFrame(data_list)
raw_data

Date,Open,High,Low,Close,Volume
str,f64,f64,f64,f64,i64
"""2025-01-03""",243.36,244.18,241.89,243.36,40244114
"""2025-01-02""",248.93,249.1,241.8201,243.85,55740731
"""2024-12-31""",252.44,253.28,249.43,250.42,39480718
"""2024-12-30""",252.23,253.5,250.75,252.2,35557542
"""2024-12-27""",257.83,258.7,253.06,255.59,42355321
…,…,…,…,…,…
"""2024-08-19""",225.72,225.99,223.04,225.89,40687813
"""2024-08-16""",223.92,226.8271,223.6501,226.05,44340240
"""2024-08-15""",224.6,225.35,222.76,224.72,46414013


In [6]:
file_path = "../../data/raw/raw_stock_data.csv"
raw_data.write_csv(file_path)